In [27]:
import numpy as np
import os
import uuid

import joblib # for persisting models

# models
from scipy.optimize import curve_fit # multip. linear regression
from sklearn.svm import SVR # support vector forrest
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import pandas as pd
import time
from datetime import datetime

In [28]:
df = pd.read_csv('../aggregated_measurements_data.csv', index_col=0)

df_cleaned = df.dropna()
print("Dropped ", len(df) - len(df_cleaned), " rows with NaN values")

Dropped  780  rows with NaN values


In [29]:
import importlib
import svr_test_bed

importlib.reload(svr_test_bed) # reload exported functions

test_sizes = []
sample_sizes = []
features = []
model_arguments = []

test_sizes.append(0.45)
sample_sizes.append(20000)
features.append(['distance', 'n_power','c_walls', 'w_walls'])
model_arguments.append({'kernel': 'rbf'})

# test_sizes.append(0.45)
# sample_sizes.append(50000)
# features.append(['distance', 'n_power','c_walls', 'w_walls'])
# model_arguments.append({'kernel': 'rbf'})

test_specification = pd.DataFrame({
    "test_size": test_sizes,
    "sample_size": sample_sizes,
    "features": features,
    "model_arguments": model_arguments,
})

# assign ids to tests
test_specification['id'] = [str(uuid.uuid4()).replace('-', '')[:8] for _ in range(len(test_specification))]

test_specification['output_file'] = './results/results.csv'

(test_results_svr, best_svr_model, best_svr_model_id) = svr_test_bed.svr_test_bed(
    data=df_cleaned,
    test_specification=test_specification,
)

# save best model
#create directory if it does not exist
os.makedirs("./best_models", exist_ok=True)
joblib.dump(best_svr_model, f"./best_models/svr_model_{best_svr_model_id}.joblib")

Test 1 of 1 with sample_size 20000, test_size 0.45 and features ['distance', 'n_power', 'c_walls', 'w_walls']
Creating Model
Fitting Model
Predicting
Test took 00:00:19
Best model id:  8059cc51  with r2:  0.9927579351204038


['./best_models/svr_model_8059cc51.joblib']